In [29]:
import ollama
import gradio as gr
from t2 import get_events_info_from_city
import json

In [30]:
MODEL = "llama3.2"

In [31]:
system_prompt = "You are EventAIde, an efficient AI assistant that helps users find events in the city the user mentioned. "
system_prompt += "Always ask user for the city and if the user doesnt mention then give any city's events. "
system_prompt += "Give only the requested event details in a short, friendly, and professional reply. "
system_prompt += "If you do not have information for a city, say so clearly."

system_message = [{"role":"system", "content": system_prompt}]

In [32]:
event_function = {
    "name": "get_events_info_from_city",
    "description": "Get the events happening from the given city. Call this whenever you need to know the events list, for example when a customer asks ‘what events are there in New York'",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "All the available events in the customer provided city",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [33]:
tools = [{"type": "function", "function": event_function}]

In [34]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = tool_call.function.arguments
    city = arguments.get('city')
    events = get_events_info_from_city(city)
    response = {
        "role": "tool",
        "content": json.dumps({"city": city,"events": events})
    }
    return response, city

In [35]:
def chatbot(message, history):
    messages = system_message+history+[{"role":"user", "content":message}]
    response = ollama.chat(model = MODEL, messages=messages, tools=tools)

    print(response)
    if response.message.tool_calls:
        message = response.message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = ollama.chat(model=MODEL, messages=messages)
        
    return response["message"]["content"]

In [36]:
gr.ChatInterface(fn = chatbot, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


model='llama3.2' created_at='2025-08-15T19:58:36.59968Z' done=True done_reason='stop' total_duration=4502060000 load_duration=3220295125 prompt_eval_count=264 prompt_eval_duration=808566458 eval_count=19 eval_duration=472183125 message=Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=[ToolCall(function=Function(name='get_events_info_from_city', arguments={'city': ''}))])
model='llama3.2' created_at='2025-08-15T19:59:18.740121Z' done=True done_reason='stop' total_duration=1309114500 load_duration=51057875 prompt_eval_count=328 prompt_eval_duration=782157083 eval_count=19 eval_duration=474488792 message=Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=[ToolCall(function=Function(name='get_events_info_from_city', arguments={'city': 'Saint Louis'}))])
